In [1]:
from utils.model_utils import name_to_model_class

model_cls, _ = name_to_model_class("GGNN")

2024-04-07 00:00:51.430945: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-07 00:00:52.351827: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


In [2]:
from utils.model_utils import name_to_task_class

task_cls, _ = name_to_task_class("PPI")

In [3]:
model_cls, task_cls

(models.ggnn_model.GGNN_Model, tasks.ppi_task.PPI_Task)

In [4]:
task_params = task_cls.default_params()
model_params = model_cls.default_params()
task_params, model_params

({'add_self_loop_edges': True,
  'tie_fwd_bkwd_edges': False,
  'out_layer_dropout_keep_prob': 1.0},
 {'max_nodes_in_batch': 50000,
  'graph_num_layers': 8,
  'graph_num_timesteps_per_layer': 1,
  'graph_layer_input_dropout_keep_prob': 1.0,
  'graph_dense_between_every_num_gnn_layers': 10000,
  'graph_model_activation_function': 'tanh',
  'graph_residual_connection_every_num_layers': 10000,
  'graph_inter_layer_norm': False,
  'max_epochs': 10000,
  'patience': 25,
  'optimizer': 'Adam',
  'learning_rate': 0.001,
  'learning_rate_decay': 0.98,
  'lr_for_num_graphs_per_batch': None,
  'momentum': 0.85,
  'clamp_gradient_norm': 1.0,
  'random_seed': 0,
  'hidden_size': 128,
  'graph_rnn_cell': 'GRU',
  'graph_activation_function': 'tanh',
  'message_aggregation_function': 'sum'})

In [5]:
task = task_cls({})

In [6]:
from dpu_utils.utils import RichPath

task_params_orig = dict(task_params)
for (param_name, param_value) in task_params.items():
    if param_name.endswith("_path"):
        task_params[param_name] = RichPath.create(param_value, None)

In [7]:
task_params

{'add_self_loop_edges': True,
 'tie_fwd_bkwd_edges': False,
 'out_layer_dropout_keep_prob': 1.0}

In [8]:

task = task_cls(task_params)
data_path = task.default_data_path()
data_path = RichPath.create(data_path, None)
task.load_data(data_path)

 Loading PPI train data from data/ppi.
 Loading PPI valid data from data/ppi.


In [9]:
random_seeds = model_params['random_seed']
if not isinstance(random_seeds, list):
    random_seeds = [random_seeds]

In [10]:
random_seeds

[0]

In [11]:
import os
import time

run_id = "_".join([task_cls.name(), model_cls.name(model_params), time.strftime("%Y-%m-%d-%H-%M-%S"), str(os.getpid())])

In [12]:
run_id

'PPI_GGNN_2024-04-07-00-01-09_8503'

In [13]:
result_dir = "./results/"

In [14]:
os.makedirs(result_dir, exist_ok=True)

In [15]:
model = model_cls(model_params, task, run_id, result_dir)


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'graph_model/gnn_layer_0/gru_cell/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/nnao45/ghq/github.com/microsoft/tf-gnn-samples/.venv/lib/python3.11/site-packages/tensorflow/python/ops/weak_tensor_ops.py", line 88, in wrapper
    return op(*args, **kwargs)  File "/home/nnao45/ghq/github.com/microsoft/tf-gnn-samples/.venv/lib/python3.11/site-packages/tensorflow/python/ops/numpy_ops/np_array_ops.py", line 316, in diag
    control_flow_assert.Assert(  File "/home/nnao45/ghq/github.com/microsoft/tf-gnn-samples/.venv/lib/python3.11/site-packages/tensorflow/python/util/traceback_utils.py", line 155, in error_handler
    del filtered_tb  File "/home/nnao45/ghq/github.com/microsoft/tf-gnn-samples/.venv/lib/python3.11/site-packages/tensorflow/python/util/dispatc

ValueError: Exception encountered when calling GRUCell.call().

[1mAttempting to slice scalar input. for '{{node graph_model/gnn_layer_0/gru_cell_1/strided_slice}} = StridedSlice[Index=DT_INT32, T=DT_INT32, begin_mask=0, ellipsis_mask=0, end_mask=0, new_axis_mask=0, shrink_axis_mask=0](graph_model/gnn_layer_0/gru_cell_1/shape_as_tensor, graph_model/gnn_layer_0/gru_cell_1/strided_slice/stack, graph_model/gnn_layer_0/gru_cell_1/strided_slice/stack_1, graph_model/gnn_layer_0/gru_cell_1/strided_slice/stack_2)' with input shapes: [], [1], [1], [1] and with computed input tensors: input[3] = <1>.[0m

Arguments received by GRUCell.call():
  • inputs=tf.Tensor(shape=(?, 128), dtype=float32)
  • states=['tf.Tensor(shape=(?, 128), dtype=float32)']
  • training=False